In [3]:
import pandas as pd
import psycopg2

In [70]:
dbname = 'football'
user = 'postgres'
password = 'TheDarkKhight'
host = 'localhost'
port = 5432

In [66]:
# Формирование тестовой выборки - случайные 90 матчей из сезона 2023/2024
data_PL23 = pd.read_csv('elo/csv/England/2023/PL.csv')
data_RFPL23 = pd.read_csv('elo/csv/Russia/2023/RFPL.csv')
all_data = data_PL23.merge(data_RFPL23, how='outer').sort_values('timestamp').reset_index(drop=True)
all_data

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,1042016,1689955200,1088,Dinamo Moscow,1,3,621,Krasnodar
1,1042021,1690020000,1084,Ural,2,1,555,CSKA Moscow
2,1042020,1690028100,2011,Nizhny Novgorod,0,2,596,Zenit Saint Petersburg
3,1042017,1690036200,597,Lokomotiv Moscow,2,2,1083,Rubin
4,1042018,1690045200,1085,Akhmat Grozny,1,2,1079,Krylya Sovetov
...,...,...,...,...,...,...,...,...
163,1035116,1696770000,39,Wolves,1,1,66,Aston Villa
164,1042096,1696771800,1088,Dinamo Moscow,0,0,1993,Fakel Voronezh
165,1035107,1696779000,42,Arsenal,1,0,50,Manchester City
166,1042094,1696780800,558,Spartak Moscow,2,2,555,CSKA Moscow


In [67]:
random_sample = all_data.sample(90).sort_values('timestamp').reset_index(drop=True)
random_sample

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,1042016,1689955200,1088,Dinamo Moscow,1,3,621,Krasnodar
1,1042021,1690020000,1084,Ural,2,1,555,CSKA Moscow
2,1042014,1690122600,558,Spartak Moscow,3,2,1080,Orenburg
3,1042019,1690131600,2012,PFC Sochi,2,0,2006,Baltika
4,1042015,1690131600,779,FC Rostov,2,1,1993,Fakel Voronezh
...,...,...,...,...,...,...,...,...
85,1042095,1696694400,621,Krasnodar,3,2,779,FC Rostov
86,1035110,1696696200,52,Crystal Palace,0,0,65,Nottingham Forest
87,1035116,1696770000,39,Wolves,1,1,66,Aston Villa
88,1042096,1696771800,1088,Dinamo Moscow,0,0,1993,Fakel Voronezh


In [79]:
other_from_2023 = pd.concat([all_data, random_sample]).drop_duplicates(keep=False).sort_values('timestamp').reset_index(drop=True)
other_from_2023

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,1042020,1690028100,2011,Nizhny Novgorod,0,2,596,Zenit Saint Petersburg
1,1042017,1690036200,597,Lokomotiv Moscow,2,2,1083,Rubin
2,1042018,1690045200,1085,Akhmat Grozny,1,2,1079,Krylya Sovetov
3,1042028,1690727400,1993,Fakel Voronezh,1,4,597,Lokomotiv Moscow
4,1042024,1690736400,621,Krasnodar,2,0,2012,PFC Sochi
...,...,...,...,...,...,...,...,...
73,1042100,1696762800,2011,Nizhny Novgorod,2,0,1079,Krylya Sovetov
74,1035108,1696770000,51,Brighton,2,2,40,Liverpool
75,1035115,1696770000,48,West Ham,2,2,34,Newcastle
76,1035107,1696779000,42,Arsenal,1,0,50,Manchester City


In [72]:
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

cursor = conn.cursor()
for _, row in random_sample.iterrows():
    cursor.execute(f'INSERT INTO public.\"test_sample_matches\" VALUES({row["fixtueID"]}, {row["home_team_id"]}, {row["away_team_id"]}, {row["home_goals"]}, {row["away_goals"]}, {row["timestamp"]}, \'test\')')

conn.commit()
cursor.close()
conn.close()

In [35]:
data_kubokRFPL22 = pd.read_csv('elo/csv/Russia/2022/KUBOK.csv')
data_kubokRFPL22 = data_kubokRFPL22[data_kubokRFPL22['home_team_id'].isin(all_data['home_team_id'].unique()) & data_kubokRFPL22['away_team_id'].isin(
                 all_data['home_team_id'].unique())]

In [44]:
data_RFPL22 = data_RFPL22.merge(data_kubokRFPL22, how='outer').sort_values('timestamp').reset_index(drop=True)
data_RFPL22.reset_index(inplace=True, drop=True)
data_RFPL22

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,879867,1657904400,1994,Khimki,1,1,596,Zenit Saint Petersburg
1,879868,1657972800,555,CSKA Moscow,2,0,1084,Ural
2,879869,1657981800,1080,Orenburg,2,4,1079,Krylya Sovetov
3,879870,1657990800,1085,Akhmat Grozny,1,1,558,Spartak Moscow
4,879871,1658059200,3985,Torpedo Moskva,1,3,2012,PFC Sochi
...,...,...,...,...,...,...,...,...
292,880104,1685800800,1080,Orenburg,3,0,1088,Dinamo Moscow
293,880105,1685800800,2011,Nizhny Novgorod,2,0,1994,Khimki
294,880106,1685800800,1084,Ural,1,0,2012,PFC Sochi
295,880101,1685800800,555,CSKA Moscow,4,1,779,FC Rostov


In [64]:
data_PL22 = pd.read_csv('elo/csv/England/2022/PL.csv')
data_kubokPL22 = pd.read_csv('elo/csv/England/2022/cup.csv')
data_kubokPL22
data_kubokPL22 = data_kubokPL22[data_kubokPL22['home_team_id'].isin(data_PL22['home_team_id'].unique()) & data_kubokPL22['away_team_id'].isin(
                 data_PL22['home_team_id'].unique())]
data_carabaoPL22 = pd.read_csv('elo/csv/England/2022/carabao.csv')
data_carabaoPL22
data_carabaoPL22 = data_carabaoPL22[data_carabaoPL22['home_team_id'].isin(data_PL22['home_team_id'].unique()) & data_carabaoPL22['away_team_id'].isin(
                 data_PL22['home_team_id'].unique())]
data_PL22 = data_PL22.merge(data_kubokPL22, how='outer')
data_PL22 = data_PL22.merge(data_carabaoPL22, how='outer')
data_PL22.sort_values('timestamp', inplace=True)
data_PL22.reset_index(inplace=True, drop=True)

In [65]:
data_PL22

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,867946,1659726000,52,Crystal Palace,0.0,2.0,42,Arsenal
1,867947,1659785400,36,Fulham,2.0,2.0,40,Liverpool
2,867948,1659794400,35,Bournemouth,2.0,0.0,66,Aston Villa
3,867949,1659794400,63,Leeds,2.0,1.0,39,Wolves
4,867951,1659794400,34,Newcastle,2.0,0.0,65,Nottingham Forest
...,...,...,...,...,...,...,...,...
405,868323,1685287800,46,Leicester,2.0,1.0,48,West Ham
406,868324,1685287800,33,Manchester United,2.0,1.0,36,Fulham
407,868316,1685287800,42,Arsenal,5.0,0.0,39,Wolves
408,868318,1685287800,55,Brentford,1.0,0.0,50,Manchester City


In [74]:
#формирование обучающей выборки (425) и валидационной выборки (90)
general_eng = data_PL22.iloc[150:410].reset_index(drop=True)
general_eng

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,868105,1668282300,39,Wolves,0.0,2.0,42,Arsenal
1,868097,1668348000,51,Brighton,1.0,2.0,66,Aston Villa
2,868098,1668357000,36,Fulham,1.0,2.0,33,Manchester United
3,972361,1671565500,34,Newcastle,1.0,0.0,35,Bournemouth
4,972362,1671739200,50,Manchester City,3.0,2.0,40,Liverpool
...,...,...,...,...,...,...,...,...
255,868323,1685287800,46,Leicester,2.0,1.0,48,West Ham
256,868324,1685287800,33,Manchester United,2.0,1.0,36,Fulham
257,868316,1685287800,42,Arsenal,5.0,0.0,39,Wolves
258,868318,1685287800,55,Brentford,1.0,0.0,50,Manchester City


In [77]:
general_rus = (data_RFPL22.iloc[120:]).reset_index(drop=True)
general_rus

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,879969,1665927000,2011,Nizhny Novgorod,0,3,596,Zenit Saint Petersburg
1,879966,1665939600,555,CSKA Moscow,2,2,558,Spartak Moscow
2,945697,1666114200,1088,Dinamo Moscow,3,1,779,FC Rostov
3,945698,1666182600,1080,Orenburg,4,2,1085,Akhmat Grozny
4,945693,1666191600,2011,Nizhny Novgorod,1,0,621,Krasnodar
...,...,...,...,...,...,...,...,...
172,880104,1685800800,1080,Orenburg,3,0,1088,Dinamo Moscow
173,880105,1685800800,2011,Nizhny Novgorod,2,0,1994,Khimki
174,880106,1685800800,1084,Ural,1,0,2012,PFC Sochi
175,880101,1685800800,555,CSKA Moscow,4,1,779,FC Rostov


In [81]:
all_general = general_eng.merge(general_rus, how='outer')
all_general = all_general.merge(other_from_2023, how='outer')
all_general.sort_values('timestamp').reset_index(drop=True, inplace=True)
all_general

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,868105,1668282300,39,Wolves,0.0,2.0,42,Arsenal
1,868097,1668348000,51,Brighton,1.0,2.0,66,Aston Villa
2,868098,1668357000,36,Fulham,1.0,2.0,33,Manchester United
3,972361,1671565500,34,Newcastle,1.0,0.0,35,Bournemouth
4,972362,1671739200,50,Manchester City,3.0,2.0,40,Liverpool
...,...,...,...,...,...,...,...,...
510,1042100,1696762800,2011,Nizhny Novgorod,2.0,0.0,1079,Krylya Sovetov
511,1035108,1696770000,51,Brighton,2.0,2.0,40,Liverpool
512,1035115,1696770000,48,West Ham,2.0,2.0,34,Newcastle
513,1035107,1696779000,42,Arsenal,1.0,0.0,50,Manchester City


In [82]:
train_data = all_general.sample(425).sort_values('timestamp').reset_index(drop=True)
valid_data = pd.concat([all_general, train_data]).drop_duplicates(keep=False).sort_values('timestamp').reset_index(drop=True)

conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

cursor = conn.cursor()
for _, row in train_data.iterrows():
    cursor.execute(f'INSERT INTO public.\"test_sample_matches\" VALUES({row["fixtueID"]}, {row["home_team_id"]}, {row["away_team_id"]}, {row["home_goals"]}, {row["away_goals"]}, {row["timestamp"]}, \'train\')')

for _, row in valid_data.iterrows():
    cursor.execute(f'INSERT INTO public.\"test_sample_matches\" VALUES({row["fixtueID"]}, {row["home_team_id"]}, {row["away_team_id"]}, {row["home_goals"]}, {row["away_goals"]}, {row["timestamp"]}, \'valid\')')
conn.commit()
cursor.close()
conn.close()